In [1]:
# THIS SCRIPT DOWNLOADS A SNAPSHOT OF THE CURRENT TRAIN DEPARTURES FOR EACH SELECTED TRAINSTART
# RUN THIS SCRIPT A FEW TIMES PER DAY TO CATCH ALL DEPATURES
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

In [2]:
import pandas as pd
stations = pd.read_csv("./generated/0_db_station_lk.csv",sep=';',encoding="utf-8")
stations.head(5)

,db_station_name,rki_landkreisname,rki_bezeichner,db_station_id,rki_ags
0,Bayreuth Hbf,Bayreuth,Kreisfreie Stadt,8000028,9462
1,Brackwede,Bielefeld,Kreisfreie Stadt,8000048,5711
2,Kaufering,Landsberg am Lech,Landkreis,8000195,9181
3,Weiden(Oberpf),Weiden i.d. OPf.,Kreisfreie Stadt,8000204,9363
4,Landshut(Bay)Hbf,Landshut,Kreisfreie Stadt,8000217,9261


In [3]:
# THIS FUNKTION EXTRACTS THE NEEDED INFORMATION FROM THE A JSON RESPONSE

def format_json_from_departureStationBoard_api(_json):
    ret = []
    
    for train in _json:
        train_entry = {}
        
        
        # ALLOW ONLY THE FOLLOWING PRODUCT TYPE
        train_entry['type'] = train['train']['type']
        if train_entry['type'] == "RB" or train_entry['type'] == "RE" or train_entry['type'] == "IC" or train_entry['type'] == "ICE":
            pass
        else:
            continue
        
        
        train_entry['name'] = train['train']['name']
        train_entry['line'] = train['train']['line']
        train_entry['number'] = train['train']['number']
        
        
        train_entry['start_station'] = train['currentStation']['id']
        train_entry['start_station'] = train['currentStation']['id']
        
        stops = ""
        for stop in train['stops']:
            #print(stop)
            stops = stops + stop['station']['id'] + ","                
        train_entry['stops'] = stops

        ret.append(train_entry)

    return ret

In [4]:
# AFTER LOADING THE STATION DATASET WHICH INCLUDES ALL STATIONS WE WANT TO ANALYSE
# THE NEXT STEP IS TO QUERY THE API FOR THE DEPATURES OF THE TRAINS AT THE SELECTED STATION
# THE RESULT SHOULD BE A LIST OF TRAINS AND THEIR DEPARTMENT STATION

import requests
import json
import time

# ALL RESULTS WILL BE STORED IN THE successful_fetch ARRAY EACH FAILED REQUEST IN failed_fetch
failed_fetch = []
successful_fetch = []

def fetch_depature_table(_station_id, _time):
    global failed_fetch
    global successful_fetch
    
    if _time is None:
        r = requests.get('https://marudor.de/api/hafas/v2/departureStationBoard?station='+str(fetch_station_id)+'&profile=db')
    else:
        r = requests.get('https://marudor.de/api/hafas/v2/departureStationBoard?station='+str(fetch_station_id)+'&profile=db,date=' + str(_time))
        
    if r.status_code == 200:
        for fr in format_json_from_departureStationBoard_api(r.json()):
            successful_fetch.append(fr)
    else:
        failed_fetch.append(fetch_station_id)


In [5]:
# FINALLY START DOWNLOADING ALL DEPATURES FOR EACH LOADED STATION

In [ ]:
stationd_id = stations['db_station_id']
for station in stationd_id:
    print(station)
    fetch_station_id = station
    fetch_station_time = None #"2021-11-25T8:00:00.000Z"
    fetch_depature_table(fetch_station_id,fetch_station_time)

8000028
8000048
8000195
8000204
8000217
8000233
8000440
8000566
8000619
8000775
8001071
8001091
8001164
8001168
8001347
8001571
8001646
8001718
8001777
8001846
8001851
8001924
8001972
8002025
8002071
8002085
8002227
8002634
8002680
8002700
8002779
8002924
8003076
8003079
8003096
8003154
8003187
8003354
8003382
8003427
8003508
8003520
8003568
8003608
8003672
8003705
8003767
8003804
8003821
8003881
8003922
8003959
8003990
8004008
8004111
8004122
8004171
8004247
8004253
8004295
8004302
8004419
8004420
8004426
8004489
8004513
8004518
8004542
8004567
8004626
8004665
8004700
8004702
8004712
8004726
8004788
8004801
8004809
8004822
8004848
8004868
8004900
8004920
8004924
8004936
8004983
8005008
8005019
8005065
8005122
8005143
8005159
8005199
8005208
8005246
8005275
8005279
8005292
8005340
8005345
8005352
8005366
8005421
8005435
8005440
8005481
8005543
8005628
8005645
8005653
8005655
8005658
8005670
8005672


In [ ]:
# SAVE RESULTS AS CSV
## UGLY !!
# TRANSFROM FETCH RESULT INTO PD COMPATIBLE DATA
_type = []
_name = []
_line = []
_number = []
_start_station = []
_stops = []
_max_capacity = []

for res in successful_fetch:
    et = None
    try:
        et = res[0]
    except:
        et = res
    
    if et is None or len(et) <= 0:
        continue
    #print(et)
    _type.append(et['type'])
    _name.append(et['name'])
    _line.append(et['line'])
    _number.append(et['number'])
    _start_station.append(et['start_station'])
    _stops.append(et['stops'])
    
    
    
    
    # ADD A ESTIMATED CAPACITY
    if et['type'] == "ICE":
        _max_capacity.append(720)
    elif et['type'] == "IC" or et['type'] == "EC":
        _max_capacity.append(468)
    elif et['type'] == "RE":
        _max_capacity.append(602)
    elif et['type'] == "RB":
        _max_capacity.append(426)


fetched_depatures = pd.DataFrame(data={'type':_type,'name':_name,'line':_line,'number':_number,'start_station':_start_station,'stops':_stops, 'max_capacity':_max_capacity})
fetched_depatures.head(5)

In [ ]:
import calendar;
import time;
ts = calendar.timegm(time.gmtime())
print(ts)

fetched_depatures.to_csv("./generated/departure_tables_raw/1_departures_table_"+str(ts)+".csv",sep=';',encoding="utf-8",index=False)